# Official data

This repo loads and analyses official data for the FNF project. As part of this, we will:

1. Load BRES and IDBR data from Nesta data_getters
2. Explore the data focusing on journalism and media
  * As part of this we need to select relevant SIC codes
3. Produce some initial outputs to be presented at the team meeting

# Preamble

In [ ]:
%run ../notebook_preamble.ipy

In [ ]:
import geopandas as gp
import seaborn as sns

In [ ]:
def create_lq(X, binary=False):
    """ Calculate the location quotient.

    Divides the share of activity in a location by the share of activity in the UK total

    Args:
        X (pandas.DataFrame): DataFrame where rows are locations, columns are sectors and values are activity in a given sector at a location.
        binary (bool, optional): If True, discretise the data with a cut-off value of 1

    Returns:
        pandas.DataFrame
    """
    Xm = X.values
    X = pd.DataFrame((Xm/Xm.sum(1)[:, np.newaxis])/(Xm.sum(0)/Xm.sum()),
            index=X.index, columns=X.columns)
    
    return (X > 1) if binary else X

### Metadata

#### Sectors names and labels

In [ ]:
journalism_sics = {'6010':'radio_broadcasting',
                   '6020':'tv_programming_broadcasting',
                   '1811':'printing_newspapers',
                   '5813':'publishing_newspapers',
                  '6391':'news_agency_activities',
                   '6312':'web_portals',}

benchmark = {'62':'computer_programming','73':'advertising',
             #'68':'real_estate'
            }

#### Colours

In [ ]:
#Create some colour lookups for different sectors.
#The idea is to have similar hues for similar sectors, and lower opacity for non-journalism sectors

colors = {x:y for x,y in zip(journalism_sics.values(),['lightcoral','red','deepskyblue','blue','cadetblue','darkorchid'])}

colors['advertising'] = 'green'
colors['journalism'] = 'blue'
colors['computer_programming'] = 'red'
colors['other'] = 'grey'
colors['real_estate'] = 'brown'

from matplotlib.colors import to_rgba

colors_rgb = {x:to_rgba(y) for x,y in colors.items()}

journ_sects = list(journalism_sics.values())+['journalism']

colors_rgb = {x:y if x in journ_sects else tuple(list(y[:3])+list((0.8,))) for x,y in colors_rgb.items()}

### Load geo metadata

In [ ]:
lad_shape = gp.read_file('../../data/external/lads/Local_Authority_Districts_December_2017_Full_Clipped_Boundaries_in_Great_Britain.shp')

In [ ]:
#A couple of the LAD names are spelt differently. Let's fix them
lad_rename_lookup = {'Rhondda Cynon Taf':'Rhondda Cynon Taff',
                 'Shepway':'Folkestone and Hythe'}


lad_shape['lad17nm'] = [lad_rename_lookup[x] if x in lad_rename_lookup.keys() else x for x in lad_shape['lad17nm']]

## Load Data

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
bres = pd.read_csv('/Users/jmateosgarcia/Desktop/mapping_innovation_scotland/data/interim/BRES_LAD_combi_5_9_2019.csv')
idbr = pd.read_csv('/Users/jmateosgarcia/Desktop/mapping_innovation_scotland/data/interim/IDBR_LAD_141_5_9_2019.csv')

In [ ]:
bres.columns = [x.strip() for x in bres.columns]
idbr.columns = [x.strip() for x in idbr.columns]

In [ ]:
def process_official(df,lookup,benchmark,id_vars=['LAD','year']):
    '''
    This function processes official data for our analysis of the state and evolution of the journalism industry in the UK
    
    Arguments:
        df is a df with a year and LAD variables we can use to group our analysis
        lookup is a sic to names lookup that we use to identify journalism related sectors
        benchmark is a lookup we use to label benchmark 2-digit sectors like 60 (computing), 64 (finance) or 68 (real estate)
    
    
    '''
    
    #Melt the data
    
    melted = pd.melt(df,id_vars=id_vars,var_name='sic_4')
    
    #print(melted.head())
    
    #Name each sector
    
    #This considers each component of journalism separately
    melted['sector'] = [lookup[x] if x in lookup.keys() else benchmark[x[:2]] if x[:2] in benchmark.keys() else 'other' for x in melted['sic_4']]
    
    #This aggregates them
    melted['sector_aggregated'] = ['journalism' if x in lookup.keys() else benchmark[x[:2]] if x[:2] in benchmark.keys() else 'other' for x in melted['sic_4']]
    
    return(melted)

def make_trend(df,sector_var,y='year',v='value'):
    '''
    
    Pandas contortions to create a linechart from the previous output
    
    Args:
        df is a long table where every row is a LAD-sector pair
        sector_var is the variable we want to visualise
        y is the year variable
        v is the value
    
    '''

    out = df.groupby([y,sector_var])[v].sum().reset_index(drop=False).pivot(index=y,columns=sector_var,
                                                                             values=v)
    
    return(out)
    
    
def plot_trend(df,ax,norm=True,smooth=3):
    
    '''
    
    Plots trends
    
    Args:
        df is a table where the rows are years and the columns are sectors
        ax is the axis for plotting
        norm is whether we want to normalise the data by year
        rolling is to smooth
    
    '''
    
    if norm==True:
        
        for c in df.columns:
            (100*df[c]/df[c].sum()).rolling(window=smooth).mean().dropna().plot(ax=ax,linewidth=3 if c in journ_sects else 1,
                                                                                              color=colors_rgb[c])
        
    else:
        
        for c in df.columns:
            (100*df[c].rolling(window=smooth).mean(),dropna()).plot(ax=ax,linewidth=3 if c in journ_sects else 1,
                                                                                              color=colors_rgb[c])
        
        
def plot_bar(df,ax,norm=False,smooth=2,journ=list(journalism_sics.values())):
    '''
    
    Args:
        df is a table where the rows are years and the columns are sectors
        ax is the ax for plotting
        norm is whether we normalise the values to share in the year or keep totals
        rolling is the window for smoothing
        jour are the columsn with journalism data
    
    '''
    
    journ_df = df[journ]
 
    
    sectors_sorted = journ_df.T.sort_values(2017,ascending=False).index    
    
    if norm==True:
        (100*journ_df[sectors_sorted].apply(lambda x: x/x.sum(),axis=1).rolling(window=smooth).mean()).dropna().plot.bar(ax=ax,stacked=True,width=0.8,
                                                                                                                        color=[colors[x] for x in sectors_sorted])
                
    else:
        (100*journ_df[sectors_sorted].rolling(window=smooth).mean()).dropna().plot.bar(ax=ax,stacked=True,width=0.8,color=[colors[x] for x in sectors_sorted])

In [ ]:
def make_lorenz(df,variable='sector',y=2017):
    '''
    Creates a lorenz curve of concentration
    
    Args:
        df is the same df as before
        y is the year to focus on
        
    
    '''
    
    focus = df.loc[df['year']==y].reset_index(drop=False).pivot_table(index='LAD',columns=variable,values='value',aggfunc='sum')
    
    #For each column we sort and normalise
    
    cumulatives = []
    
    for c in focus.columns:
        
        cumulatives.append(focus[c].sort_values(ascending=False)/focus[c].sum())
        
        
    out = pd.concat(cumulatives,axis=1)
    out.columns = focus.columns
    
        
    return(out)
    
def plot_lorenz(df,ax):
    '''
    Plots the lorenz curve (share of activity accounted by locations in various positions of the ranking)
        
    
    '''
    
    sorted_cont = []
    
    for c in df.columns:
        
        sorted_cont.append(df[c].sort_values(ascending=False).cumsum().reset_index(drop=True))
        
    sorted_cont = pd.concat(sorted_cont,axis=1)
    
    
    
    sorted_cont.columns = df.columns
    
    sorted_sectors = sorted_cont.loc[5].sort_values().index[::-1]
    
    for x in sorted_sectors:
        (100*sorted_cont[x]).plot(ax=ax,alpha=0.75,color=colors_rgb[x],
                               linewidth=3 if x in journ_sects else 1)
        

def plot__histo_lorenz(df_1,df_2):
    '''
    Plots the lorenz curve (share of activity accounted by locations in various positions of the ranking) comparing t1 and t2
        
    
    '''
    
    fig,ax = plt.subplots(figsize=(16,5),ncols=len(df_1.columns),sharey=True)
    
    sectors = df_2.iloc[10].sort_values().index
    
    sorted_cont = []
    
    for n,c in enumerate(sectors):
        
        df_1[c].sort_values(ascending=False).cumsum().reset_index(drop=True).plot(ax=ax[n],linewidth=2,color='red',alpha=0.75)
        df_2[c].sort_values(ascending=False).cumsum().reset_index(drop=True).plot(ax=ax[n],linewidth=2,color='blue',alpha=0.75)
        
        
        ax[n].set_title(re.sub('_','\n',c),size=10)
        
        ax[n].legend(labels=['2009','2017'])
        

    

In [ ]:
def widen_proc(df,year,variable,normalize=False,make_geo=False,geo_option=lad_shape):
    '''
    Function that takes a processsed df (as above and widens it). We can then concatenate it with the shapefile for plotting
    
    Args:
        df is the dataframe
        year is the year to focus on. 
        variable is the variable we want in the columns of the df
        normalize is whether we want to output LQs or totals
    
    '''
    
    #We widen a sector df so that every column is a sector 
    
    wide = pd.pivot_table(df.loc[df['year']==year],index='LAD',columns=variable,values='value',aggfunc='sum')
    
    if normalize == True:
        wide = create_lq(wide)
        
    if make_geo!=False:
        
        #Merges the sector df with the shapefile
        wide =geo_option.merge(wide.reset_index(drop=False),left_on='lad17nm',right_on='LAD')
        
    return(wide)
    

def map_sector(geodf,sector,title,ax,**kwargs):
    '''
    Map a sector
    
    Args:
        geodf is the geo_df
        sector is the sector
        ax is the ax
        kwargs are to control the plot
        
    
    '''
    
    geodf.plot(column=sector,ax=ax,**kwargs)
    
    ax.set_title(title)
    
    ax.axis('off')


def year_comp(act_df,variable,sector,ax,years=[2009,2017],**kwargs):
    '''
    
    Plots two maps comparing years
    
    Args:
        act df is the df to compare
        sector is the sector
        years are the years to compare
        
    '''
    
    #Extract the two years
    y1,y2 = [widen_proc(act_df,year=y,variable=variable,normalize=True,make_geo=True) for y in years]

    #fig,ax = plt.subplots(figsize=(7,10))

    map_sector(y1,sector,title='_'.join([sector,str(years[0])]),ax=ax[0],**kwargs)
    
    map_sector(y2,sector,title='_'.join([sector,str(years[1])]),ax=ax[1],**kwargs)

def sect_comp(act_df,variable,sectors,ax,year=2017,**kwargs):
    '''
    
    Compares various sectors
    
    
    
    '''
    
    #Extract the two years
    activity = widen_proc(act_df,year=year,variable=variable,normalize=True,make_geo=True)


    for n,sector in enumerate(sectors):
        map_sector(activity,sector,title=sector,ax=ax[n],**kwargs)
    
    
    
    

In [ ]:
def sectoral_report(df,plot_destination,year_lims = [2009,2017],proc_kwargs={'lookup':journalism_sics,
                                                                             'benchmark':benchmark},
                    map_kwargs={'scheme':'Fisher_Jenks','cmap':'viridis','edgecolor':'grey','linewidth':0,'legend':True}):
    '''
    
    Performs a series of routine analyses of official data for the FNF project.
    
    Args:
        plot_destination (str): location to store the plots
        year_lims (list) years for historical comparisons
        proc_kwargs (dict) parameters to process the data
        map_kwargs (dict) parameters to visualise the data in maps
    
    '''
    
    #Process data
    proc = process_official(df,**proc_kwargs)
    
    ##############
    ##Trend charts
    ##############
    print('making trends')
    
    #Create trend charts
    trend_micro,trend_agg = [make_trend(proc,sector_var=v) for v in ['sector','sector_aggregated']]
    
    #Plot aggregated trends
    fig,ax = plt.subplots(figsize=(9,5))

    plot_trend(trend_agg,ax=ax)

    ax.legend(bbox_to_anchor=(1,1))
    ax.set_title('Aggregate trends')
    ax.set_ylabel('% in year')
    plt.tight_layout()

    
    plt.savefig(f'../../data/processed/{plot_destination}/{today_str}_aggregated_trends.pdf')
    
    
    #Plot micro trends
    fig,ax = plt.subplots(figsize=(9,5))

    plot_trend(trend_micro,ax=ax)

    ax.legend(bbox_to_anchor=(1,1))
    ax.set_title('Detailed trends')
    ax.set_ylabel('% in year')
    plt.tight_layout()
    
    plt.savefig(f'../../data/processed/{plot_destination}/{today_str}_micro_trends.pdf')
    
    
    #Plot bars
    fig,ax = plt.subplots(figsize=(9,5))

    plot_bar(trend_micro,ax,norm=True)

    ax.legend(bbox_to_anchor=(1,1))
    ax.set_title('Composition of journalism')
    ax.set_ylabel('% in year')
    plt.tight_layout()
    
    plt.savefig(f'../../data/processed/{plot_destination}/{today_str}_micro_bar.pdf')
    
    
    ###############
    #Lorenz charts
    ###############
    print('making concentration')
    
    
    y0 = year_lims[0]
    y1 = year_lims[1]
    
    shares_early,shares_late = [make_lorenz(proc,y=year) for year in [y0,y1]]
    
    fig,ax = plt.subplots(figsize=(9,6),sharey=True)

    plot_lorenz(shares_late,ax)

    ax.legend(bbox_to_anchor=(1,1))
    ax.set_title('Concentration by sector')
    plt.tight_layout()
    
    plt.savefig(f'../../data/processed/{plot_destination}/{today_str}_lorenz_last.pdf')
    
    plot__histo_lorenz(shares_early,shares_late)
    plt.tight_layout()
    plt.savefig(f'../../data/processed/{plot_destination}/{today_str}_lorenz_change.pdf')
    
    ###########
    #Maps
    ###########
    print('making maps')
    
    #Map of journalism
    fig,ax = plt.subplots(figsize=(14,10),ncols=2)

    year_comp(proc,'sector_aggregated','journalism',years=year_lims,ax=ax,**plot_kwargs)
    
    plt.tight_layout()
    plt.savefig(f'../../data/processed/{plot_destination}/{today_str}_map_year_comp.png')


    
    #Map comparing journalism with other sectors
    fig,ax = plt.subplots(figsize=(21,10),ncols=3)

    sect_comp(proc,'sector_aggregated',['advertising','computer_programming','journalism'],ax=ax,**plot_kwargs)
    plt.tight_layout()
    
    plt.savefig(f'../../data/processed/{plot_destination}/{today_str}_map_sector_comp_agg.png')
    
    #Map comparing newspaper publishing and web portals
    fig,ax = plt.subplots(figsize=(14,10),ncols=2)

    sect_comp(proc,'sector',['publishing_newspapers','web_portals'],ax=ax,**plot_kwargs)
    plt.tight_layout()
    
    plt.savefig(f'../../data/processed/{plot_destination}/{today_str}_map_newspapers_portals.png')
    
    #Map comparing radio and TV broadcasting
    fig,ax = plt.subplots(figsize=(14,10),ncols=2)

    sect_comp(proc,'sector',['tv_programming_broadcasting','radio_broadcasting'],ax=ax,**plot_kwargs)
    plt.tight_layout()
    
    plt.savefig(f'../../data/processed/{plot_destination}/{today_str}_map_tv_radio.png')
    
    ###########
    #Heatmap
    ###########
    print('making colocation')
    
    sector_spec = widen_proc(proc,2017,'sector',normalize=True)

    fig,ax = plt.subplots(figsize=(9,7))
    
    sns.heatmap(drop_diagonal(sector_spec.corr()),cmap='bwr',center=0)
    ax.set_title('Sector colocation')
    plt.tight_layout()
    
    plt.savefig(f'../../data/processed/{plot_destination}/{today_str}_sector_colocation.pdf')
    
    

In [ ]:
def drop_diagonal(corr):
    '''
    Utility to drop diagonal in a correlation matrix so we can visualise it as a heatmap
    
    '''
    
    sector_corr_array = np.array(corr)

    np.fill_diagonal(sector_corr_array,np.nan)

    out = pd.DataFrame(sector_corr_array,index=corr.index,columns=corr.columns)

    return(out)

### a. Generate report

#### BRES report

In [ ]:
plot_kwargs = {'scheme':'Fisher_Jenks','cmap':'viridis','edgecolor':'grey','linewidth':0,'legend':True}

In [ ]:
sectoral_report(bres,plot_destination='../../reports/figures/research_slides/bres')

#### A couple of additional statistics

In [ ]:
bres_proc = process_official(bres,journalism_sics,benchmark)
bres_proc.groupby(['year','sector_aggregated'])['value'].sum().reset_index(drop=False).pivot(index='year',columns='sector_aggregated',values='value')

In [ ]:
bres_proc.groupby(['year','sector'])['value'].sum().reset_index(drop=False).pivot(index='year',columns='sector',values='value')

In [ ]:
make_lorenz(bres_proc)['publishing_newspapers'].sort_values(ascending=False).iloc[:10].sum()

In [ ]:
make_lorenz(bres_proc)['other'].sort_values(ascending=False).iloc[:10].sum()

In [ ]:
make_lorenz(bres_proc)['web_portals'].sort_values(ascending=False).iloc[:10].sum()

In [ ]:
57/13

#### IDBR report

In [ ]:
sectoral_report(idbr,plot_destination='../../reports/figures/research_slides/idbr',year_lims=[2010,2017])

### Combine with secondary data

We combine industrial data reported above with secondary data

In [ ]:
secondary = pd.read_csv('../../data/processed/13_9_2019_secondary_data.csv',index_col=0)

In [ ]:
def secondary_comp(secondary,df,plot_destination,proc_kwargs,activity_threshold=0.25):
    '''
    
    Here we combine secondary LAD-level data with industry data and analyse their correlations
    
    Args:
        secondary is a df with secondary data
        df is a df with industry data (could be bres or idbr)
    
    '''
    
    #Calculate sector specialisation
    
    print('making clustering')
    
    #First we need to process the data
    proc = process_official(df,**proc_kwargs)
    
    #Then calculate specialisations
    sector_spec = widen_proc(proc,2017,'sector',normalize=True)
    
    #Merge with the secondary data
    combined = pd.concat([sector_spec,secondary],axis=1)
    
    #Plot clustermap
    #fig,ax = plt.subplots(12,12)
    
    sns.clustermap(combined.corr(),cmap='bwr',figsize=(6,6)
                   #ax=ax
                  )
    
    #plt.tight_layout()
    plt.savefig(f'../../data/processed/{plot_destination}/{today_str}_secondary_clustering.pdf')
    
    
    #Identify LADs with low levels of journalism employment
    
    print('checking low activity areas')
    journalism_spec = widen_proc(proc,2017,'sector_aggregated',normalize=False)['journalism']
    
    #Low quantile
    quant = journalism_spec.quantile(activity_threshold)
    
    print(quant)
    
    #Create a dummy if value below quantile
    journalism_low = journalism_spec<quant
    
    #Add this to the data
    sec_j = pd.concat([sec,journalism_low],axis=1)

    #Compare high w/ low areas
    comparison = pd.concat([sec_j.groupby('journalism')[x].median() for x in secondary.columns],axis=1).iloc[:,2:]
    
    fig,ax = plt.subplots(figsize=(8,5))
    
    (100*(comparison.loc[True]/comparison.loc[False])-100).sort_values(ascending=True).plot.barh(color='blue',ax=ax)
    
    ax.set_title('Profile: areas with low journalism density')
    plt.tight_layout()
    
    plt.savefig(f'../../data/processed/{plot_destination}/{today_str}_profile_comparison.pdf')
    
    

In [ ]:
secondary_comp(secondary,bres,'../../reports/figures/research_slides/bres/',proc_kwargs={'lookup':journalism_sics,
                                                                             'benchmark':benchmark})

In [ ]:
journalism_sics_filter = journalism_sics.copy()

del journalism_sics_filter['1811']

In [ ]:
secondary_comp(secondary,idbr,'../../reports/figures/research_slides/idbr/',proc_kwargs={'lookup':journalism_sics_filter,
                                                                             'benchmark':benchmark},activity_threshold=0.5)

### Measure number of journalists per capita

We use longitudinal population data from Nomis

In [ ]:
lad_pop = pd.read_csv('../../data/processed/13_9_2019_lad_pop_longitudinal.csv')

In [ ]:
bres_pop_merged = pd.merge(bres_proc,lad_pop,left_on=['LAD','year'],right_on=['geography_name','date_name'])

In [ ]:
bres_pop_merged['sector_share_pop'] = bres_pop_merged['value']/bres_pop_merged['obs_value']

journ_share_pop = bres_pop_merged.pivot_table(index=['LAD','year'],columns='sector_aggregated',values='sector_share_pop')['journalism'].reset_index(
    drop=False).pivot_table(index='LAD',columns='year',values='journalism').sort_values(2017,ascending=False)

In [ ]:
ax = (1000*journ_share_pop[[2009,2017]]).plot(figsize=(20,5))

#ax.set_xticks(np.arange(len(journ_share_pop.index)))
#ax.set_xticklabels(journ_share_pop.index,rotation=90,size=8)
